In [77]:
import json
import requests
from lxml import html
from bs4 import BeautifulSoup
from collections import OrderedDict
import argparse
import re
import dateutil.parser
import pandas as pd
import time
from selenium import webdriver
from textatistic import Textatistic
import pickle
import numpy as np
from selenium.common.exceptions import TimeoutException



## Pull articles list from home page

## loop through pickle list of people

In [85]:
with open("all_the_people_j.pkl", 'rb') as picklefile: 
    all_the_people = pickle.load(picklefile)


In [87]:
people_sample = list(all_the_people)
len(people_sample)

50

In [43]:
master_articles = []

In [44]:
def get_home_articles(soup):
    article_urls = []
    try:
        articles = soup.find_all('a', {'data-action':'open-post'})
        for item in articles:
            if item.text == 'Read more…':
                url = (item['href'])
                url = re.sub('\?source(.*)','',url)
                article_urls.append(url)
        return article_urls
    except: []

In [45]:
driver = webdriver.Chrome(executable_path="/Users/mayamidzik/tools/chromedriver")

for url in people_sample:
    url= url+'/latest'
    driver.get(url)
    
    #scroll to bottom of page
    SCROLL_PAUSE_TIME = 0.1
    
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait to load page
        time.sleep(SCROLL_PAUSE_TIME)

        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height
    
    #get the html for the page
    innerHTML = driver.execute_script("return document.body.innerHTML")
    soup = BeautifulSoup(innerHTML,"lxml")
    
    articles = get_home_articles(soup)
    master_articles.extend(articles)

    

In [46]:
len(master_articles)

272

# loop through master_articles list to get attributes

## first define functions for attributes

In [62]:
#check if the article is associated with a publication
def check_pub(soup):
    if soup.find_all('a',{'class':'js-collectionLogoOrName'}):
        return 1
    else: return 0

In [63]:
#get_writer

def get_author(soup):
    author_url = soup.find('a',{'data-action-source':'post_header_lockup'})
    if author_url:
        author_url = author_url['href']
        author_url = re.sub('\?source(.*)','',author_url)
        return author_url
    else: return np.nan

In [64]:
#get the date and time of the article:
#should write an exception in case time is missing
def get_datetime(soup):
    published = soup.find('time')
    if published: 
        return published['datetime']
    else: return np.nan

def get_date(soup):
    published = soup.find('time')
    if published: 
        return published.text
    else: return np.nan

In [65]:
#get the reading time for the article

def get_reading_time(soup):
    reading = soup.find('span',{'class':'readingTime'})
    if reading:
        reading = reading['title']
        #reading = list(filter(str.isdigit, reading))
        #return int(''.join(reading))
        return reading
    else: return np.nan

In [66]:
#get the tags for the article
def get_tags(soup):
    tag_list = []
    tags = articlesoup.find('ul',{'class':'tags tags--postTags tags--borderless'})
    if tags:
        tags = tags.find_all('li')
        for tag in tags:
            tag_list.append(tag.text)
    return tag_list

In [67]:
#get claps
def get_claps(soup):
    claps = articlesoup.find('button',{'data-action':'show-recommends'})
    if claps: return claps.text
    else: return np.nan

In [68]:


def get_meta_traits(soup):
    article_traits = {}
    currentURL =  driver.current_url
    article_traits[currentURL] = {}
    article_traits[currentURL] = {'publisher': check_pub(articlesoup)}
    article_traits[currentURL].update({'author': get_author(articlesoup)})
    article_traits[currentURL].update({'datetime': get_datetime(articlesoup)})
    article_traits[currentURL].update({'date': get_date(articlesoup)})
    article_traits[currentURL].update({'reading_time': get_reading_time(articlesoup)})
    article_traits[currentURL].update({'claps': get_claps(articlesoup)})
    return article_traits


In [69]:
#get all attributes around the article text

def get_text_traits(soup):
    article_text_traits= {}
    #get the number of images and add to dictionary
    article_images = soup.find_all('img',{'class' : 'progressiveMedia-image js-progressiveMedia-image'})
    if article_images:
        #add to dictionary
        article_text_traits['images'] = len(article_images)
    else: article_text_traits['images'] = np.nan
    
    #get the tota; text block for the article
    article_text = articlesoup.find_all('p')
    total_text = ''
    if article_text:
        for text in article_text:
            total_text+=(text.text)
            
        #get total word count
        article_text_traits['words'] = len(total_text.split())
        
        #get sentence count
        article_text_traits['sentences'] = total_text.count('.')
        
        #get flesch readability
        try:
            s = Textatistic(total_text)
            article_text_traits['flesch'] = s.flesch_score
        except: article_text_traits['flesch'] = np.nan
            
    else:
        article_text_traits['words'] = np.nan
        article_text_traits['sentences'] = np.nan
        article_text_traits['flesch'] = np.nan
    
    return article_text_traits



In [79]:
master_article_traits = {}
#if have closed driver
#driver = webdriver.Chrome(executable_path="/Users/mayamidzik/tools/chromedriver")

for url in master_articles:
    driver.get(url)
    try:
        articleHTML = driver.execute_script("return document.body.innerHTML")
        articlesoup = BeautifulSoup(articleHTML,"lxml")

        currentURL =  driver.current_url
        article_meta_traits = get_meta_traits(articlesoup)
        article_text_traits = get_text_traits(articlesoup)
        article_meta_traits[currentURL].update(article_text_traits)
        master_article_traits.update(article_meta_traits)
    except TimeoutException:
        pass
    

In [80]:
len(master_article_traits)

272

In [84]:
with open('my_circle_articles_dict.pkl', 'wb') as picklefile:
    pickle.dump(master_article_traits, picklefile)

In [81]:
#write to a dataframe
df = pd.DataFrame.from_dict(master_article_traits).T
df.reset_index(level=0, inplace=True)
df['index'] = df
df.rename(columns = {'index':'url'},inplace = True)
df

,url,author,claps,date,datetime,flesch,images,publisher,reading_time,sentences,words
0,https://blog.foam.space/foam-the-importance-of...,https://blog.foam.space/@foamspace,513,Apr 10,2018-04-11T03:04:49.034Z,47.6973,7,1,12 min read,121,2608
1,https://blog.foam.space/foam-token-curated-reg...,https://blog.foam.space/@foamspace,399,Apr 2,2018-04-03T05:09:33.655Z,56.8932,4,1,9 min read,73,1598
2,https://blog.foam.space/introduction-to-proof-...,https://blog.foam.space/@foamspace,2K,Jan 23,2018-01-23T15:01:03.377Z,48.5909,3,1,8 min read,87,1868
3,https://blog.foam.space/the-spatial-index-9793...,https://blog.foam.space/@foamspace,1K,Feb 7,2018-02-07T15:01:01.103Z,53.0229,6,1,5 min read,44,905
4,https://blog.mapbox.com/bring-realistic-styles...,https://blog.mapbox.com/@Mapbox,272,Apr 13,2018-04-13T15:27:15.242Z,67.3515,2,1,2 min read,13,184
5,https://blog.mapbox.com/google-andrew-chen-map...,https://blog.mapbox.com/@Mapbox,152,Apr 17,2018-04-17T14:10:14.541Z,59.5618,1,1,3 min read,15,348
6,https://blog.mapbox.com/indoor-maps-with-mapwi...,https://blog.mapbox.com/@Mapbox,702,Apr 12,2018-04-12T17:11:00.855Z,61.911,1,1,2 min read,21,381
7,https://blog.mapbox.com/mapbox-unity-sdk-augme...,https://blog.mapbox.com/@Mapbox,214,Apr 13,2018-04-13T19:58:56.085Z,75.7558,5,1,3 min read,23,443
8,https://blog.mapbox.com/mapping-donor-network-...,https://blog.mapbox.com/@Mapbox,53,Apr 16,2018-04-16T18:30:42.249Z,55.6379,3,1,3 min read,22,382
9,https://blog.mapbox.com/maps-for-machines-loca...,https://blog.mapbox.com/@Mapbox,75,Apr 12,2018-04-12T15:08:46.470Z,56.8575,3,1,3 min read,19,388


In [83]:
df.to_pickle('my_circle_articles.pkl')